In [54]:
from openbabel import pybel
import pandas as pd
from pathlib import Path

In [55]:
def read_file(format, filename):
    pybel.ob.obErrorLog.SetOutputLevel(0)
    mols = list(pybel.readfile(format, filename))
    return (mols)

In [87]:
def build_dataframe_of_extracted_metadata(mol_list, group_str):
    data, mol_name, smile, GROUP_TYPE = [],[],[],[]
    for line in mol_list:
        data.append(line.data)
        mol_name.append(line.data["NAME"])
        GROUP_TYPE.append(line.data[group_str])
        smile.append(line.data["SMILES"])
    df = pd.DataFrame([mol_name, GROUP_TYPE, smile])
    return (df)

In [58]:
def rearrange_dataframe(df):
    df = df.transpose()
    df.columns = ["NAME", "GROUP_TYPE", "SMILE"] 
    return (df)

In [60]:
def inputdata_reformatted(format, file, group_str):
    mol_list = read_file(format, file)
    df = build_dataframe_of_extracted_metadata(mol_list, group_str)
    df = rearrange_dataframe(df) 
    return (df)

In [82]:
#file = Path("RECETOX_GC-EI-MS_20201028.sdf")
file = Path("sample.sdf")
format = file.suffix[1:]
group_str = "Class"
my_df = inputdata_reformatted(format, file.name, group_str)
my_df = my_df.set_index("SMILE")

In [83]:
group_df = my_df.groupby("GROUP_TYPE")

In [84]:
dic_g = group_df.groups
keys = list(dic_g.keys())

In [86]:
for k in keys:
    dic_val = dic_g[k].values
    outfile = open(group_str + '_' + k +'.smi', 'w+')
    for x in range(len(dic_val)):
        outfile.write(dic_val[x] + '\n')
    outfile.close()